# D1 — Detecção de Anomalias em Telemetria (Médio)

Detecte anomalias em telemetria mista de robôs. Retorne escores de anomalia por linha.

### Contexto
Robôs transmitem telemetria de motores, IMUs, sensores térmicos e nós de computação. Falhas se manifestam como desvios sutis que devem ser sinalizados antes de ultrapassarem limites de segurança. Simulamos telemetria mista via [`common.data_utils.telemetry_anomaly_dataset`](common/data_utils.py): normais seguem uma Gaussiana de média zero, enquanto anomalias adicionam saliências correlacionadas entre canais para emular travamentos de atuadores ou eventos de superaquecimento.

## Especificação de Entrada e Saída
**Entrada:** `X (N×d)` telemetria  
**Saída:** `scores (N,)` escores de anomalia (maior = mais anômalo)

Orientações de runtime/recursos: qubits ≤ 10, passos do otimizador ≤ 150; mantenha a inferência por lote abaixo de ~0,05 s.

### Dicas de solução
- Ajuste um modelo nominal usando apenas inliers (ex.: QSVM de uma classe com [`common.quantum_utils.feature_map`](common/quantum_utils.py)) e pontue pela distância à fronteira aprendida.
- Caminho de autoencoder quântico: codifique vetores de telemetria em um latente de baixa dimensão com um circuito variacional; o erro de reconstrução vira o escore de anomalia.
- Calibre limiares de decisão em telemetria limpa de validação para que escores altos correspondam a alertas acionáveis.

_Orientação de baseline:_ [`common.baselines.baseline_anomaly_threshold`](common/baselines.py) atinge AUROC ≈ 0.92–0.94 em seeds variadas; testes ocultos requerem AUROC ≥ 0.90.

## Setup

In [ ]:
import sys
import os
# Adiciona o diretório pai (qhacka-2025) ao caminho do Python
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import numpy as np
from common import data_utils as du
from common import baselines as bl
from common import quantum_utils as qu
import pennylane as qml
from pennylane import numpy as pnp
np.random.seed(1337)

## Baseline (referência)

In [4]:
X, y = du.telemetry_anomaly_dataset(n=600, d=8, seed=0, anomaly_rate=0.03)
scores = bl.baseline_anomaly_threshold(X)
from sklearn.metrics import roc_auc_score
au = roc_auc_score(y, scores)
print("AUROC do baseline:", au)


AUROC do baseline: 0.998663612065674


## Sua tarefa: implementar `solve(...)`

In [ ]:
def solve(X):
    
    # --- Início da Solução D1 ---
    
    # 1. Imports
    import pennylane as qml
    from pennylane import numpy as pnp
    from common import quantum_utils as qu
    from sklearn.decomposition import PCA 

    # --- 2. Compressão Híbrida (PCA Clássico) ---
    n_qubits = 3 
    pca = PCA(n_components=n_qubits)
    X_pca = pca.fit_transform(X)

    # --- 3. Preparação dos Dados Quânticos (pós-PCA) ---
    N = len(X_pca)
    norms = pnp.linalg.norm(X_pca, axis=1, keepdims=True)
    max_norm = pnp.max(norms)
    if max_norm == 0: max_norm = 1.0 
    X_norm = (X_pca / max_norm) * pnp.pi 

    # --- 4. Definição do QAE ---
    n_latent = 1 
    trash_wires = list(range(n_latent, n_qubits))
    dev = qu.default_device(n_qubits=n_qubits) 
    n_layers = 1
    params_shape = qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=n_qubits)

    @qml.qnode(dev)
    def qae_circuit(params, x_in):
        qml.AngleEmbedding(x_in, wires=range(n_qubits), rotation='X')
        qml.StronglyEntanglingLayers(params, wires=range(n_qubits))
        return qml.probs(wires=trash_wires) 

    def cost_fn(params, x_in):
        probs = qae_circuit(params, x_in)
        return 1.0 - probs[0] 
    
    def batch_cost_fn(params, X_batch):
        cost = 0.0
        for x_in in X_batch:
            cost += cost_fn(params, x_in)
        return cost / len(X_batch)

    # --- 5. Treinamento ---
    steps = 2
    
    batch_size = 32
    opt = qml.AdamOptimizer(stepsize=0.01)
    params = pnp.random.normal(0, 0.1, size=params_shape, requires_grad=True)

    for i in range(steps):
        batch_indices = pnp.random.randint(0, N, (batch_size,))
        X_batch = X_norm[batch_indices]
        params = opt.step(lambda p: batch_cost_fn(p, X_batch), params)

    # --- 6. Inferência (Pontuação) ---
    scores = pnp.zeros(len(X_norm))
    for i, x_in in enumerate(X_norm):
        scores[i] = cost_fn(params, x_in)
        
    return scores.unwrap()

# --- Fim da Solução D1 ---

## Testes públicos (rápidos)

In [14]:
from sklearn.metrics import roc_auc_score
X, y = du.telemetry_anomaly_dataset(n=800, d=8, seed=1, anomaly_rate=0.03)
scores = solve(X)
au = roc_auc_score(y, scores)
print("AUROC público:", au)
assert scores.shape == (len(y),)
assert au >= 0.90, "AUROC público abaixo do limiar"
print("OK")


AUROC público: 1.0
OK


> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.

In [15]:
# --- CÉLULA DE VALIDAÇÃO (seed=99) ---
print("--- Iniciando Teste de Validação (Robustedez) ---")
from sklearn.metrics import roc_auc_score

# Usar um seed diferente (ex: 99) para simular os testes ocultos da banca
X_val, y_val = du.telemetry_anomaly_dataset(n=800, d=8, seed=99, anomaly_rate=0.03)

# Rodar a mesma solução 'solve'
scores_val = solve(X_val)
au_val = roc_auc_score(y_val, scores_val)

print("AUROC Validação (seed=99):", au_val)

# Pegar o AUROC do teste público (variável 'au' da célula anterior)
try:
    print("AUROC Público   (seed=1):", au) 
    print(f"Diferença: {abs(au - au_val):.4f}")
    assert abs(au - au_val) < 0.05, "Overfitting detectado! A diferença é muito alta."
    print("VEREDITO: Solução robusta, não houve overfitting.")
except NameError:
    print("Execute a célula de Teste Público primeiro para comparar.")

print("--- Fim do Teste de Validação ---")

--- Iniciando Teste de Validação (Robustedez) ---
AUROC Validação (seed=99): 1.0
AUROC Público   (seed=1): 1.0
Diferença: 0.0000
VEREDITO: Solução robusta, não houve overfitting.
--- Fim do Teste de Validação ---
